# Array Processing Fundamentals

## Tutorial 1: Uniform Linear Arrays and Signal Models

This notebook introduces the fundamental concepts of array signal processing for Direction of Arrival (DOA) estimation. We'll cover:

1. **Uniform Linear Array (ULA) Geometry**
2. **Signal Model for Narrowband Sources**
3. **Array Manifold and Steering Vectors**
4. **Spatial Sampling and the Visible Region**
5. **Data Generation for DOA Estimation**

---

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

from doa_methods.array_processing import UniformLinearArray, SignalModel
from doa_methods.utils.visualization import DOAPlotter

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plotter = DOAPlotter()

print("Setup completed successfully!")

## 1. Uniform Linear Array (ULA) Geometry

A Uniform Linear Array consists of $M$ identical sensors placed at equal intervals along a straight line. The key parameters are:

- **$M$**: Number of array elements
- **$d$**: Inter-element spacing (typically in wavelengths)
- **$f_c$**: Carrier frequency
- **$\lambda$**: Wavelength ($\lambda = c/f_c$)

### Creating a ULA

In [ ]:
# Create a 16-element ULA with half-wavelength spacing
array = UniformLinearArray(M=16, d=0.5, fc=1e9)  # 1 GHz carrier

# Display array information
info = array.get_info()
print("Array Information:")
for key, value in info.items():
    print(f"  {key}: {value}")

# Visualize array geometry
fig, ax = plt.subplots(figsize=(14, 4))
plotter.plot_array_geometry(array, ax=ax)
plt.tight_layout()
plt.show()

### Why Half-Wavelength Spacing?

The choice of $d = \lambda/2$ is crucial to avoid **spatial aliasing**:

- For $d \leq \lambda/2$: No spatial aliasing (unambiguous DOA estimation)
- For $d > \lambda/2$: Spatial aliasing occurs (ambiguous angles)

Let's explore the effect of element spacing:

In [ ]:
# Compare different element spacings
spacings = [0.3, 0.5, 0.8, 1.2]  # In wavelengths
theta_test = np.deg2rad(30)  # Test angle: 30 degrees

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, d in enumerate(spacings):
    # Create array with this spacing
    test_array = UniformLinearArray(M=8, d=d)
    
    # Compute directivity pattern
    angle_grid = np.linspace(-np.pi/2, np.pi/2, 181)
    directivity = test_array.steering_vector(theta_test).conj().T @ test_array.array_manifold(angle_grid)
    directivity = np.abs(directivity.flatten())**2 / test_array.M**2
    
    axes[i].plot(np.rad2deg(angle_grid), 10*np.log10(directivity + 1e-10))
    axes[i].axvline(np.rad2deg(theta_test), color='red', linestyle='--', label='Steering Direction')
    axes[i].set_title(f'd = {d}λ')
    axes[i].set_xlabel('Angle (degrees)')
    axes[i].set_ylabel('Directivity (dB)')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()
    
    # Highlight spatial aliasing
    if d > 0.5:
        axes[i].text(0.02, 0.98, 'Spatial Aliasing!', transform=axes[i].transAxes,
                    verticalalignment='top', color='red', fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.suptitle('Effect of Element Spacing on Array Directivity', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 2. Signal Model for Narrowband Sources

The received signal at the array is modeled as:

$$\mathbf{x}(t) = \mathbf{A}(\boldsymbol{\theta}) \mathbf{s}(t) + \mathbf{n}(t)$$

where:
- $\mathbf{x}(t)$: $M \times 1$ received signal vector
- $\mathbf{A}(\boldsymbol{\theta})$: $M \times K$ array manifold matrix
- $\mathbf{s}(t)$: $K \times 1$ source signal vector
- $\mathbf{n}(t)$: $M \times 1$ additive noise vector

### Array Manifold and Steering Vectors

The **steering vector** for angle $\theta$ is:

$$\mathbf{a}(\theta) = [1, e^{-j\omega}, e^{-j2\omega}, \ldots, e^{-j(M-1)\omega}]^T$$

where $\omega = 2\pi \frac{d}{\lambda} \sin(\theta)$ is the **spatial frequency**.

In [ ]:
# Demonstrate steering vectors for different angles
angles_deg = [-30, -15, 0, 15, 30]
angles_rad = np.deg2rad(angles_deg)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot steering vector magnitude and phase
for i, (angle_deg, angle_rad) in enumerate(zip(angles_deg, angles_rad)):
    a = array.steering_vector(angle_rad)
    element_indices = np.arange(array.M)
    
    # Magnitude (should be 1 for all elements)
    ax1.plot(element_indices, np.abs(a), 'o-', label=f'{angle_deg}°', 
             color=plotter.colors[i], markersize=6)
    
    # Phase
    ax2.plot(element_indices, np.angle(a), 's--', label=f'{angle_deg}°', 
             color=plotter.colors[i], markersize=6, alpha=0.8)

ax1.set_xlabel('Element Index')
ax1.set_ylabel('Magnitude')
ax1.set_title('Steering Vector Magnitude')
ax1.grid(True, alpha=0.3)
ax1.legend()

ax2.set_xlabel('Element Index')
ax2.set_ylabel('Phase (radians)')
ax2.set_title('Steering Vector Phase')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Show the relationship between angle and spatial frequency
angle_range = np.linspace(-np.pi/2, np.pi/2, 181)
spatial_freqs = array.spatial_frequency(angle_range)

plt.figure(figsize=(10, 6))
plt.plot(np.rad2deg(angle_range), spatial_freqs/(2*np.pi), 'b-', linewidth=2)
plt.xlabel('Angle (degrees)')
plt.ylabel('Normalized Spatial Frequency (cycles)')
plt.title('Spatial Frequency vs Angle for ULA')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='Nyquist Limit')
plt.axhline(y=-0.5, color='red', linestyle='--', alpha=0.7)
plt.legend()
plt.show()

## 3. The Visible Region

For a ULA, the **visible region** is defined by the physical constraints:

$$-\frac{\pi}{2} \leq \theta \leq \frac{\pi}{2}$$

This corresponds to spatial frequencies:

$$-\pi \frac{d}{\lambda} \leq \omega \leq \pi \frac{d}{\lambda}$$

For $d = \lambda/2$, the spatial frequency range is $[-\pi, \pi]$, which perfectly matches the digital signal processing convention.

In [ ]:
# Visualize the visible region concept
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Different array configurations
configs = [
    {'M': 8, 'd': 0.5, 'title': '8 elements, d=0.5λ'},
    {'M': 16, 'd': 0.5, 'title': '16 elements, d=0.5λ'},
    {'M': 8, 'd': 0.3, 'title': '8 elements, d=0.3λ'},
    {'M': 8, 'd': 0.8, 'title': '8 elements, d=0.8λ'}
]

axes = axes.flatten()

for i, config in enumerate(configs):
    test_array = UniformLinearArray(M=config['M'], d=config['d'])
    
    # Array manifold over visible region
    angle_grid = test_array.angle_grid(N_grid=181)
    A = test_array.array_manifold(angle_grid)
    
    # Plot array manifold magnitude for first few elements
    for elem in range(min(4, config['M'])):
        axes[i].plot(np.rad2deg(angle_grid), np.abs(A[elem, :]), 
                    label=f'Element {elem}')
    
    axes[i].set_title(config['title'])
    axes[i].set_xlabel('Angle (degrees)')
    axes[i].set_ylabel('|Array Manifold|')
    axes[i].grid(True, alpha=0.3)
    axes[i].legend()
    axes[i].set_xlim(-90, 90)

plt.tight_layout()
plt.show()

## 4. Signal Generation and Noise Models

The `SignalModel` class provides tools for generating synthetic data with known DOAs, which is essential for algorithm development and testing.

In [ ]:
# Create signal model
signal_model = SignalModel(array)

# Define source parameters
doas_true = np.deg2rad([-20, 15])  # Two sources at -20° and +15°
N_snapshots = 200
snr_db = 10

# Generate synthetic data
X, S, N = signal_model.generate_signals(
    doas=doas_true,
    N_snapshots=N_snapshots,
    snr_db=snr_db,
    signal_type='complex_sinusoid',
    seed=42  # For reproducible results
)

print(f"Generated data shapes:")
print(f"  Received signals X: {X.shape} (M × N)")
print(f"  Source signals S: {S.shape} (K × N)")
print(f"  Noise N: {N.shape} (M × N)")
print(f"  True DOAs: {np.rad2deg(doas_true):.1f} degrees")

In [ ]:
# Visualize the generated data
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Source signals (time domain)
time_indices = np.arange(min(100, N_snapshots))  # Show first 100 samples
for k in range(len(doas_true)):
    axes[0, 0].plot(time_indices, np.real(S[k, time_indices]), 
                   label=f'Source {k+1} (Real)', linewidth=1.5)
    axes[0, 0].plot(time_indices, np.imag(S[k, time_indices]), 
                   '--', label=f'Source {k+1} (Imag)', alpha=0.7)

axes[0, 0].set_title('Source Signals (Time Domain)')
axes[0, 0].set_xlabel('Time Index')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Received signals at first few elements
for m in range(min(4, array.M)):
    axes[0, 1].plot(time_indices, np.real(X[m, time_indices]), 
                   label=f'Element {m}', alpha=0.7)

axes[0, 1].set_title('Received Signals (Real Part)')
axes[0, 1].set_xlabel('Time Index')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Noise at first element
axes[0, 2].plot(time_indices, np.real(N[0, time_indices]), 'gray', alpha=0.7, label='Real')
axes[0, 2].plot(time_indices, np.imag(N[0, time_indices]), 'gray', alpha=0.5, linestyle='--', label='Imag')
axes[0, 2].set_title('Noise (Element 0)')
axes[0, 2].set_xlabel('Time Index')
axes[0, 2].set_ylabel('Amplitude')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Sample covariance matrix
R = signal_model.sample_covariance(X)
plotter.plot_covariance_matrix(R, ax=axes[1, 0], title='Sample Covariance Matrix')

# Eigenvalues of covariance matrix
eigenvals = np.linalg.eigvals(R)
eigenvals = np.sort(np.real(eigenvals))[::-1]  # Sort in descending order

axes[1, 1].semilogy(eigenvals, 'bo-', markersize=6)
axes[1, 1].axvline(len(doas_true) - 0.5, color='red', linestyle='--', 
                  alpha=0.7, label='Signal/Noise Subspace Boundary')
axes[1, 1].set_title('Covariance Matrix Eigenvalues')
axes[1, 1].set_xlabel('Eigenvalue Index')
axes[1, 1].set_ylabel('Eigenvalue Magnitude')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

# Signal-to-noise ratio visualization
signal_power = np.mean(np.abs(S)**2, axis=1)
noise_power = np.mean(np.abs(N)**2)
snr_measured = 10 * np.log10(signal_power / noise_power)

axes[1, 2].bar(range(len(doas_true)), snr_measured, alpha=0.7, 
              color=['blue', 'orange'][:len(doas_true)])
axes[1, 2].axhline(snr_db, color='red', linestyle='--', label=f'Target SNR ({snr_db} dB)')
axes[1, 2].set_title('Measured SNR per Source')
axes[1, 2].set_xlabel('Source Index')
axes[1, 2].set_ylabel('SNR (dB)')
axes[1, 2].set_xticks(range(len(doas_true)))
axes[1, 2].grid(True, alpha=0.3)
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 5. Effects of Array Parameters

Let's explore how different array parameters affect the received signal characteristics.

In [ ]:
# Compare different array sizes
array_sizes = [4, 8, 16, 32]
test_doas = np.deg2rad([-15, 15])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, M in enumerate(array_sizes):
    # Create array
    test_array = UniformLinearArray(M=M, d=0.5)
    test_signal_model = SignalModel(test_array)
    
    # Generate data
    X_test, _, _ = test_signal_model.generate_signals(
        doas=test_doas, N_snapshots=100, snr_db=10, seed=42)
    
    # Compute and plot covariance eigenvalues
    R_test = test_signal_model.sample_covariance(X_test)
    eigenvals_test = np.sort(np.real(np.linalg.eigvals(R_test)))[::-1]
    
    axes[i].semilogy(eigenvals_test, 'bo-', markersize=4)
    axes[i].axvline(len(test_doas) - 0.5, color='red', linestyle='--', alpha=0.7)
    axes[i].set_title(f'M = {M} elements')
    axes[i].set_xlabel('Eigenvalue Index')
    axes[i].set_ylabel('Eigenvalue')
    axes[i].grid(True, alpha=0.3)
    
    # Add text with array aperture
    aperture = (M - 1) * 0.5  # in wavelengths
    axes[i].text(0.02, 0.98, f'Aperture: {aperture}λ', transform=axes[i].transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

plt.suptitle('Effect of Array Size on Eigenvalue Structure', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Key Takeaways

From this tutorial, you should understand:

1. **ULA Geometry**: Half-wavelength spacing prevents spatial aliasing
2. **Signal Model**: The linear relationship between sources, array response, and measurements
3. **Steering Vectors**: How the array responds to signals from different directions
4. **Spatial Sampling**: The relationship between physical angles and spatial frequencies
5. **Data Characteristics**: How array size, SNR, and source configuration affect the received data

## Next Steps

In the next tutorial, we'll explore **Classical DOA Methods** (Conventional Beamforming and Capon), which use the concepts introduced here to estimate source directions.

---

### Exercises

Try these exercises to reinforce your understanding:

1. **Spatial Aliasing**: Create a ULA with $d = \lambda$ and observe spatial aliasing effects
2. **Resolution**: Compare the eigenvalue separation for closely vs. widely spaced sources
3. **SNR Effects**: Generate data with different SNR values and observe the impact on eigenvalues
4. **Correlation**: Generate correlated sources and examine the effect on the covariance matrix


In [ ]:
# Exercise playground - try your own experiments here!

# Example: Experiment with spatial aliasing
aliasing_array = UniformLinearArray(M=8, d=1.0)  # Full wavelength spacing

# Your code here...
